In [2]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader,TensorDataset
import time
strat = time.perf_counter()#高性能计时器，用于性能测试

#读取训练数据和测试数据
o_train = pd.read_csv('./train.csv')
o_test = pd.read_csv('./test.csv')

#自己的数据集，需要对原始数据进行处理
#原数据 第一列是序号， 从第二列到导数第二列都是 维度，最后一列是房价
#对各维度的预处理(标准化)方式：数值型的转为[-1,1]之间 z-score 标准化，新数据=（原数据-均值）/标准差
#非数值型中的  无序型进行独热编码(one-hot encoding)，有序型 自己定义其数值 转换为数值型  本数据集默认全部为无序型
#空值：每一个特征的全局平均值来代替无效值

#将训练集与测试集的特征数据合并在一起 统一进行处理
#loc：通过行标签索引数据 iloc：通过行号索引行数据 ix：通过行标签或行号索引数据（基于loc和iloc的混合）

#合并数据集
all_features = pd.concat((o_train.loc[:,'Area':'Neighborhood'],o_test.loc[:,'Area':'Neighborhood']))
all_labels = pd.concat((o_train.loc[:,'Price'],o_test.loc[:,'Price']))
#特征列从'Area'到'Neighborhood'，标签列是'Price'
#训练集和测试机的数据进行合并
#对特征值进行数据预处理
#取出所有的数值型特征名称
numeric_feats = all_features.dtypes[all_features.dtypes != "object"].index#数值类型
object_feats = all_features.dtypes[all_features.dtypes == "object"].index#非数值类型

# 将数值型特征进行 z-score 标准化
all_features[numeric_feats] = all_features[numeric_feats].apply(lambda x: (x - x.mean()) / (x.std()))#标准化值 = (原始值 - 均值) / 标准差

#对无序型进行one-hot encoding
all_features = pd.get_dummies(all_features,prefix=object_feats, dummy_na=True)

#空值：每一个特征的全局平均值来代替无效值 NA就是指空值
all_features = all_features.fillna(all_features.mean())

#对标签进行数据预处理
#对标签进行 z-score 标准化
mean = all_labels.mean()#计算均值
std = all_labels.std()#计算标准差
all_labels = (all_labels - mean)/std#标准化

num_train = o_train.shape[0]
train_features = all_features[:num_train].values.astype(np.float32)#(1314, 331)
test_features = all_features[num_train:].values.astype(np.float32)#(146, 331)
train_labels = all_labels[:num_train].values.astype(np.float32)
test_labels = all_labels[num_train:].values.astype(np.float32)#将合并后的训练集和测试集进行分离
#至此 输入数据准备完毕 可以看见 经过one-hot编码后  特征维度增加了很多 81->331

#数据类型转换，数组转换成张量
train_features = torch.from_numpy(train_features)
train_labels = torch.from_numpy(train_labels).unsqueeze(1)
test_features = torch.from_numpy(test_features)
test_labels = torch.from_numpy(test_labels).unsqueeze(1)
train_set = TensorDataset(train_features,train_labels)
test_set = TensorDataset(test_features,test_labels)

#设置迭代器
train_data = DataLoader(dataset=train_set,batch_size=64,shuffle=True)
test_data  = DataLoader(dataset=test_set,batch_size=64,shuffle=False)

#设置网络结构
class Net(torch.nn.Module):# 继承 torch 的 Module
    def __init__(self, n_feature, n_output):
        super(Net, self).__init__()     # 继承 __init__ 功能
        # 定义每层用什么样的形式
        self.layer1 = torch.nn.Linear(n_feature, 600)   #
        self.layer2 = torch.nn.Linear(600, 1200)   #
        self.layer3 = torch.nn.Linear(1200, n_output)

    def forward(self, x):   # 这同时也是 Module 中的 forward 功能
        x = self.layer1(x)
        x = torch.relu(x)      #
        x = self.layer2(x)
        x = torch.relu(x)      #
        x = self.layer3(x)
        return x
net = Net(44,1)

#反向传播算法 SGD Adam等
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
#均方损失函数
criterion =	torch.nn.MSELoss()

#记录用于绘图
losses = []#记录每次迭代后训练的loss
eval_losses = []#测试的

for i in range(100):
    train_loss = 0
    # train_acc = 0
    net.train() #网络设置为训练模式 暂时可加可不加
    for tdata,tlabel in train_data:
        #前向传播
        y_ = net(tdata)
        #记录单批次一次batch的loss
        loss = criterion(y_, tlabel)
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #累计单批次误差
        train_loss = train_loss + loss.item()

    losses.append(train_loss / len(train_data))
    # 测试集进行测试 只用于进行评估
    eval_loss = 0
    net.eval()  # 可加可不加
    for edata, elabel in test_data:
        # 前向传播
        y_ = net(edata)
        # 记录单批次一次batch的loss，测试集就不需要反向传播更新网络了
        loss = criterion(y_, elabel)
        # 累计单批次误差
        eval_loss = eval_loss + loss.item()
    eval_losses.append(eval_loss / len(test_data))

    print('训练次数: {}, 训练集损失: {}, 测试集损失: {}'.format(i, train_loss / len(train_data), eval_loss / len(test_data)))

#测试最终模型的精准度，测试集的平均误差
y_ = net(test_features)
y_pre = y_ * std + mean#反求房价
print('测试集预测值：',y_pre.squeeze().detach().cpu().numpy())
print('模型平均误差：',abs(y_pre - (test_labels*std + mean)).mean().cpu().item() )
end =time.perf_counter()
print('模型运行时间：',end - strat)

训练次数: 0, 训练集损失: 0.9101608267852238, 测试集损失: 0.6884080767631531
训练次数: 1, 训练集损失: 0.7692265482175917, 测试集损失: 0.6205475926399231
训练次数: 2, 训练集损失: 0.6517136707192376, 测试集损失: 0.5835627317428589
训练次数: 3, 训练集损失: 0.5466629082248324, 测试集损失: 0.5429505705833435
训练次数: 4, 训练集损失: 0.47889794196401325, 测试集损失: 0.5359954237937927
训练次数: 5, 训练集损失: 0.4190204654421125, 测试集损失: 0.542162299156189
训练次数: 6, 训练集损失: 0.38366842837560744, 测试集损失: 0.5662252306938171
训练次数: 7, 训练集损失: 0.36919859974157243, 测试集损失: 0.5897507071495056
训练次数: 8, 训练集损失: 0.347181926880564, 测试集损失: 0.5980970859527588
训练次数: 9, 训练集损失: 0.34501995146274567, 测试集损失: 0.6149763464927673
训练次数: 10, 训练集损失: 0.3317767151054882, 测试集损失: 0.6111993789672852
训练次数: 11, 训练集损失: 0.32333513420252574, 测试集损失: 0.6160455942153931
训练次数: 12, 训练集损失: 0.321102595045453, 测试集损失: 0.6543641686439514
训练次数: 13, 训练集损失: 0.3129508133445467, 测试集损失: 0.6387500166893005
训练次数: 14, 训练集损失: 0.3132401981524059, 测试集损失: 0.6211321353912354
训练次数: 15, 训练集损失: 0.30806732177734375, 测试集损失: 0.6151152849197388